In [1]:
import os
os.environ.setdefault('AWS_PROFILE', 'ghactivity')

'ghactivity'

In [3]:
!aws s3 rm s3://aigithub/landing/ghactivity/ --recursive

In [4]:
import boto3

In [5]:
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('jobs')
table.delete_item(Key={'job_id': 'ghactivity_ingest'})
item = {
    'job_id': 'ghactivity_ingest',
    'job_description': 'Ingest ghactivity data to s3',
    'is_active': 'Y',
    'baseline_days': 3
}
table.put_item(Item=item)

{'ResponseMetadata': {'RequestId': '0DUF7AG28KNSB45I6K23Q31CA7VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 08 Jun 2022 14:49:33 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '0DUF7AG28KNSB45I6K23Q31CA7VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [6]:
def get_job_details(job_name):
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('jobs')
    job_details = table.get_item(Key={'job_id': job_name})['Item']
    return job_details

In [7]:
get_job_details('ghactivity_ingest')    

{'job_description': 'Ingest ghactivity data to s3',
 'is_active': 'Y',
 'job_id': 'ghactivity_ingest',
 'baseline_days': Decimal('3')}

In [8]:
from datetime import datetime as dt
from datetime import timedelta as td
import time

In [16]:
def get_next_file_name(job_details):
    job_start_time = int(time.mktime(dt.now().timetuple()))
    job_run_bookmark_details = job_details.get('job_run_bookmark_details')
    baseline_days = int(job_details['baseline_days'])
    if job_run_bookmark_details:
        dt_part = job_run_bookmark_details['last_run_file_name'].split('.')[0]
        next_file_name = f"{dt.strftime(dt.strptime(dt_part, '%Y-%m-%d-%H') + td(hours=1), '%Y-%m-%d-%-H')}.json.gz"
    else:
        next_file_name = f'{dt.strftime(dt.now().date() - td(days=baseline_days), "%Y-%m-%d")}-0.json.gz'
    return job_start_time, next_file_name

In [17]:
job_details = get_job_details('ghactivity_ingest')

In [18]:
job_start_time, next_file = get_next_file_name(job_details)

In [19]:
job_start_time

1654700142

In [15]:
next_file

'2022-06-05-0.json.gz'

In [23]:
import requests

In [24]:
bucket_name = 'aigithub'

In [25]:
folder = 'landing/ghactivity'

In [29]:
def upload_file_to_s3(file_name, bucket_name, folder):
   print(f'Getting the {file_name} from gharchive')
   res = requests.get(f'https://data.gharchive.org/{file_name}')

   print(f'Uploading {file_name} to s3 under s3://{bucket_name}/{folder}')
   s3_client = boto3.client('s3')
   upload_res = s3_client.put_object(
      Bucket=bucket_name,
      Key=f'{folder}/{file_name}',
      Body=res.content
   )

   return {
      'last_run_file_name': f's3://{bucket_name}/{folder}/{file_name}',
      'status_code': upload_res['ResponseMetadata']['HTTPStatusCode']
   }


In [30]:
job_run_details = upload_file_to_s3(next_file, bucket_name, folder)

Getting the 2022-06-05-0.json.gz from gharchive
Uploading 2022-06-05-0.json.gz to s3 under s3://aigithub/landing/ghactivity


In [31]:
def save_job_run_details(job_details, job_run_details, job_start_time):
    dynamodb = boto3.resource('dynamodb')
    job_run_details_item = {
        'job_id': job_details['job_id'],
        'job_run_time': job_start_time,
        'job_run_bookmark_details': job_run_details,
        'create_ts': int(time.mktime(dt.now().timetuple()))
    }
    job_run_details_table = dynamodb.Table('job_run_details')
    job_run_details_table.put_item(Item=job_run_details_item)
    
    job_details_table = dynamodb.Table('jobs')
    job_details['job_run_bookmark_details'] = job_run_details
    job_details_table.put_item(Item=job_details)

In [32]:
save_job_run_details(job_details, job_run_details, job_start_time)

In [33]:
!aws s3 ls s3://aigithub/landing/ghactivity/

2022-06-08 20:38:37   33989636 2022-06-05-0.json.gz
